# Imports and Config

In [1]:
import os
from typing import Optional, Dict, Tuple
import numpy as np
import warnings

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random

from scipy.optimize import minimize_scalar
import pandas as pd

SEED = 99
NUM_RUNS = 50

os.makedirs("results", exist_ok=True)

# `TheoreticalBenchmarks`

In [2]:
class TheoreticalBenchmarks:
    """Calculate theoretical Nash and Monopoly prices/profits for market models"""
    
    def __init__(self, seed=None):
        self.seed = seed
        if seed is not None:
            np.random.seed(seed)
    
    def calculate_logit_benchmarks(self, shock_cfg=None):
        """
        Calculate Nash and Monopoly benchmarks for Logit model
        With shocks: Uses Monte Carlo integration
        Without shocks: Uses analytical formulas
        """
        # Model parameters
        a = 2.0
        c = 1.0
        mu = 0.25
        a0 = 0.0
        
        if shock_cfg is None or not shock_cfg.get('enabled', False):
            # No shocks - use standard benchmarks
            p_N = 1.473
            p_M = 1.925
            
            # Calculate profits at these prices
            exp_N = np.exp((a - p_N) / mu)
            exp_outside = np.exp(a0 / mu)
            q_N = exp_N / (2 * exp_N + exp_outside)
            E_pi_N = (p_N - c) * q_N
            
            exp_M = np.exp((a - p_M) / mu)
            q_M = exp_M / (2 * exp_M + exp_outside)
            E_pi_M = (p_M - c) * q_M
            
            return {
                'p_N': p_N,
                'E_pi_N': E_pi_N,
                'p_M': p_M,
                'E_pi_M': E_pi_M,
                'shock_enabled': False
            }
        
        # With shocks - use Monte Carlo
        scheme_params = {
            'A': {'rho': 0.3, 'sigma_eta': 0.5},
            'B': {'rho': 0.95, 'sigma_eta': 0.05},
            'C': {'rho': 0.9, 'sigma_eta': 0.3}
        }
        
        scheme = shock_cfg.get('scheme', 'A')
        params = scheme_params[scheme.upper()]
        rho = params['rho']
        sigma_eta = params['sigma_eta']
        
        # Unconditional variance of shocks
        sigma2 = sigma_eta**2 / (1 - rho**2)
        sigma = np.sqrt(sigma2)
        
        # Monte Carlo samples
        N = 10000
        shock_mode = shock_cfg.get('mode', 'independent')
        
        if shock_mode == 'independent':
            xi_samples = np.random.normal(0, sigma, (N, 2))
        else:  # correlated
            xi_samples = np.random.normal(0, sigma, N)
        
        # Expected profit functions
        def E_pi1(p1, p2):
            if shock_mode == 'independent':
                exps1 = np.exp((a - p1 + xi_samples[:, 0]) / mu)
                exps2 = np.exp((a - p2 + xi_samples[:, 1]) / mu)
            else:
                exps1 = np.exp((a - p1 + xi_samples) / mu)
                exps2 = np.exp((a - p2 + xi_samples) / mu)
            den = exps1 + exps2 + np.exp(a0 / mu)
            qs = exps1 / den
            return np.mean((p1 - c) * qs)
        
        def E_pi2(p1, p2):
            if shock_mode == 'independent':
                exps1 = np.exp((a - p1 + xi_samples[:, 0]) / mu)
                exps2 = np.exp((a - p2 + xi_samples[:, 1]) / mu)
            else:
                exps1 = np.exp((a - p1 + xi_samples) / mu)
                exps2 = np.exp((a - p2 + xi_samples) / mu)
            den = exps1 + exps2 + np.exp(a0 / mu)
            qs = exps2 / den
            return np.mean((p2 - c) * qs)
        
        # Nash equilibrium
        def best_response(p_j):
            def neg_E_pi(p_i):
                return -E_pi1(p_i, p_j)
            res = minimize_scalar(neg_E_pi, bounds=(c, c + 5), method='bounded')
            return res.x
        
        p_guess = 1.5
        for _ in range(50):
            p_guess = best_response(p_guess)
        p_N = p_guess
        E_pi_N = E_pi1(p_N, p_N)
        
        # Monopoly
        def neg_E_joint(p):
            return -(E_pi1(p, p) + E_pi2(p, p))
        
        res_M = minimize_scalar(neg_E_joint, bounds=(c, c + 5), method='bounded')
        p_M = res_M.x
        E_pi_M = E_pi1(p_M, p_M)
        
        return {
            'p_N': p_N,
            'E_pi_N': E_pi_N,
            'p_M': p_M,
            'E_pi_M': E_pi_M,
            'shock_enabled': True,
            'scheme': scheme,
            'mode': shock_mode,
            'sigma': sigma
        }
    
    def calculate_hotelling_benchmarks(self, shock_cfg=None):
        """
        Calculate Nash and Monopoly benchmarks for Hotelling model
        Shocks don't affect expected benchmarks (linearity in net shock)
        """
        # Model parameters
        c = 0.0
        v_bar = 1.75
        theta = 1.0
        
        # Standard benchmarks (unchanged with shocks)
        p_N = 1.00
        p_M = 1.25
        
        # Calculate profits
        # At Nash: both firms charge 1, split market equally
        q_N = 0.5
        E_pi_N = p_N * q_N
        
        # At Monopoly: both charge 1.25, split market equally
        q_M = 0.5
        E_pi_M = p_M * q_M
        
        return {
            'p_N': p_N,
            'E_pi_N': E_pi_N,
            'p_M': p_M,
            'E_pi_M': E_pi_M,
            'shock_enabled': shock_cfg is not None and shock_cfg.get('enabled', False),
            'note': 'Shocks do not affect expected benchmarks for Hotelling'
        }
    
    def calculate_linear_benchmarks(self, shock_cfg=None):
        """
        Calculate Nash and Monopoly benchmarks for Linear model
        Shocks don't affect expected benchmarks (linearity in shocks)
        """
        # Model parameters
        c = 0.0
        a_bar = 1.0
        d = 0.25
        
        # Standard benchmarks (unchanged with shocks)
        p_N = 0.4286
        p_M = 0.5
        
        # Calculate profits
        denominator = 1 - d**2
        
        # At Nash
        q_N = (a_bar - p_N - d * (a_bar - p_N)) / denominator
        E_pi_N = p_N * q_N
        
        # At Monopoly
        q_M = a_bar - p_M
        E_pi_M = p_M * q_M
        
        return {
            'p_N': p_N,
            'E_pi_N': E_pi_N,
            'p_M': p_M,
            'E_pi_M': E_pi_M,
            'shock_enabled': shock_cfg is not None and shock_cfg.get('enabled', False),
            'note': 'Shocks do not affect expected benchmarks for Linear'
        }
    
    def calculate_all_benchmarks(self, shock_cfg=None):
        """Calculate benchmarks for all three models"""
        results = {
            'logit': self.calculate_logit_benchmarks(shock_cfg),
            'hotelling': self.calculate_hotelling_benchmarks(shock_cfg),
            'linear': self.calculate_linear_benchmarks(shock_cfg)
        }
        return results
    
    def generate_benchmark_table(self, shock_configs):
        """
        Generate a comprehensive table of benchmarks for different configurations
        
        Args:
            shock_configs: List of shock configurations to test
        
        Returns:
            pandas DataFrame with benchmarks
        """
        data = []
        
        for config in shock_configs:
            config_name = config.get('name', 'No Config')
            benchmarks = self.calculate_all_benchmarks(config)
            
            for model, bench in benchmarks.items():
                data.append({
                    'Configuration': config_name,
                    'Model': model.upper(),
                    'Nash Price': round(bench['p_N'], 4),
                    'Nash Profit': round(bench['E_pi_N'], 4),
                    'Monopoly Price': round(bench['p_M'], 4),
                    'Monopoly Profit': round(bench['E_pi_M'], 4),
                    'Shock Enabled': bench['shock_enabled']
                })
        
        return pd.DataFrame(data)


# `MarketEnv` Class

In [3]:
class MarketEnv:
    """Base Market environment supporting all three models with optional shocks"""
   
    def __init__(
        self,
        market_model: str = "logit",
        shock_cfg: Optional[Dict] = None,
        n_firms: int = 2,
        horizon: int = 10000,
        seed: Optional[int] = None
    ):
        self.model = market_model.lower()
        self.n_firms = n_firms
        self.N = 15 # Number of discrete prices
        self.horizon = horizon
        self.t = 0
        self.rng = np.random.RandomState(seed)
       
        # Model-specific parameters (verified against PDFs)
        if self.model == "logit":
            self.c = 1.0
            self.a = 2.0
            self.a0 = 0.0
            self.mu = 0.25
            self.P_N = 1.473
            self.P_M = 1.925
        elif self.model == "hotelling":
            self.c = 0.0
            self.v_bar = 1.75
            self.theta = 1.0
            self.P_N = 1.00
            self.P_M = 1.25
        elif self.model == "linear":
            self.c = 0.0
            self.a_bar = 1.0
            self.d = 0.25
            
            # Nash (Cournot duopoly)
            self.P_N = self.a_bar * (1 - self.d) / (2 - self.d)  # 0.4286
            q_N = self.P_N  # By symmetry at equilibrium
            pi_N = self.P_N * q_N  # 0.1959
            
            # Monopoly (single firm, no competition)
            self.P_M = self.a_bar / 2  # 0.5
            q_M = self.a_bar - self.P_M  # 0.5 (residual demand, NOT Cournot!)
            pi_M = self.P_M * q_M  # 0.25
            
            # Store for Delta calculations
            self.pi_N_linear = pi_N  # 0.1959
            self.pi_M_linear = pi_M  # 0.25
            
            # Verify profit range is healthy
            profit_range = pi_M - pi_N  # Should be ~0.05
            if profit_range < 0.04:
                import warnings
                warnings.warn(f"Linear model profit range too small: {profit_range:.4f}")
        else:
            raise ValueError(f"Unknown model: {self.model}")
       
        # Construct price grid
        span = self.P_M - self.P_N
        self.price_grid = np.linspace(
            self.P_N - 0.15 * span,
            self.P_M + 0.15 * span,
            self.N
        )
       
        # Initialize shock configuration
        self.shock_cfg = shock_cfg or {}
        self.shock_enabled = self.shock_cfg.get("enabled", False)
       
        if self.shock_enabled:
            self.shock_mode = self.shock_cfg.get("mode", "correlated")
            scheme = self.shock_cfg.get("scheme", None)
           
            # Get AR(1) parameters (verified against PDFs)
            if scheme:
                scheme_params = {
                    'A': {'rho': 0.3, 'sigma_eta': 0.5},
                    'B': {'rho': 0.95, 'sigma_eta': 0.05},
                    'C': {'rho': 0.9, 'sigma_eta': 0.3}
                }
                if scheme.upper() in scheme_params:
                    params = scheme_params[scheme.upper()]
                    self.rho = params['rho']
                    self.sigma_eta = params['sigma_eta']
                else:
                    raise ValueError(f"Unknown scheme: {scheme}")
            else:
                self.rho = self.shock_cfg.get('rho', 0.9)
                self.sigma_eta = self.shock_cfg.get('sigma_eta', 0.15)
           
            # Initialize shock generators
            if self.shock_mode == "independent":
                self.shock_generators = [
                    AR1_Shock(self.rho, self.sigma_eta, seed=seed+i if seed else None)
                    for i in range(self.n_firms)
                ]
            else: # correlated
                self.shock_generator = AR1_Shock(self.rho, self.sigma_eta, seed=seed)
           
            self.current_shocks = np.zeros(self.n_firms)
       
        self.reset()
   
    def reset(self):
        """Reset environment - returns state as actual prices"""
        self.t = 0
       
        # Reset shocks
        if self.shock_enabled:
            if self.shock_mode == "independent":
                for gen in self.shock_generators:
                    gen.reset()
            else:
                self.shock_generator.reset()
            self.current_shocks = np.zeros(self.n_firms)
       
        # Initialize prices at middle of grid
        mid_idx = self.N // 2
        self.current_price_idx = np.array([mid_idx] * self.n_firms)
       
        return self._get_state()
   
    def _get_state(self):
        """Get current state as actual prices (numpy array)"""
        return self.price_grid[self.current_price_idx].copy()
    
    def _get_state_indices(self):
        """Get current state as price indices (for Q-learning compatibility)"""
        return tuple(self.current_price_idx)
   
    def _generate_shocks(self):
        """Generate next period shocks"""
        if not self.shock_enabled:
            return np.zeros(self.n_firms)
       
        if self.shock_mode == "independent":
            shocks = np.array([gen.generate_next() for gen in self.shock_generators])
        else:
            shock = self.shock_generator.generate_next()
            shocks = np.array([shock] * self.n_firms)
       
        return shocks
   
    def get_current_shocks(self):
        """Get current shocks (for PSO evaluation)"""
        return self.current_shocks.copy() if self.shock_enabled else np.zeros(self.n_firms)
   
    def calculate_demand_and_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Calculate demand and profit given prices and shocks"""
        if self.model == "logit":
            return self._logit_demand_profit(prices, shocks)
        elif self.model == "hotelling":
            return self._hotelling_demand_profit(prices, shocks)
        elif self.model == "linear":
            return self._linear_demand_profit(prices, shocks)
   
    def _logit_demand_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Logit demand - shocks affect variance but not Nash equilibrium
        
        Critical: E[demand | shocks] = demand(no shocks) since E[ε] = 0
        Nash price must remain constant regardless of shock scheme
        """
        # Base utilities WITHOUT shocks (defines Nash equilibrium)
        base_utilities = (self.a - prices) / self.mu
        
        # Add shocks OUTSIDE mu scaling to preserve E[utility]
        realized_utilities = base_utilities + shocks
        
        # Numerical stability
        max_util = np.max(realized_utilities)
        exp_utils = np.exp(realized_utilities - max_util)
        exp_outside = np.exp(self.a0/self.mu - max_util)
        
        denominator = np.sum(exp_utils) + exp_outside
        demands = exp_utils / denominator
        profits = (prices - self.c) * demands
        
        return demands, profits
   
    def _hotelling_demand_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Hotelling demand with net shock affecting boundary (verified against PDF)"""
        p1, p2 = prices[0], prices[1]
        epsilon_net = shocks[0] - shocks[1] if self.n_firms == 2 else 0
       
        x_hat = 0.5 + (p2 - p1 + epsilon_net) / (2 * self.theta)
        x_hat = np.clip(x_hat, 0, 1)
       
        q1 = x_hat
        q2 = 1 - x_hat
        demands = np.array([q1, q2])
        profits = prices * demands
       
        return demands, profits
   
    def _linear_demand_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Linear demand with additive shocks (verified against PDF)"""
        a_shocked = self.a_bar + shocks
        denominator = 1 - self.d**2
       
        q1 = ((a_shocked[0] - prices[0]) - self.d * (a_shocked[1] - prices[1])) / denominator
        q2 = ((a_shocked[1] - prices[1]) - self.d * (a_shocked[0] - prices[0])) / denominator
       
        q1 = max(0, q1)
        q2 = max(0, q2)
        demands = np.array([q1, q2])
        profits = prices * demands
       
        return demands, profits
   
    def step(self, action_indices):
        """Execute one step - returns (state_prices, profits, done, info)
        
        Args:
            action_indices: Array of price indices for each agent
            
        Returns:
            next_state: Numpy array of actual prices (not indices)
            profits: Numpy array of profits for each agent
            done: Episode termination flag
            info: Dictionary with prices, demands, and shocks
        """
        action_indices = np.asarray(action_indices, dtype=int)
       
        # Update price indices
        self.current_price_idx = action_indices
        prices = self.price_grid[self.current_price_idx]
       
        # Generate shocks for this period
        self.current_shocks = self._generate_shocks()
       
        # Calculate demands and profits
        demands, profits = self.calculate_demand_and_profit(prices, self.current_shocks)
       
        # Update time
        self.t += 1
       
        # Get next state (actual prices, not indices)
        next_state = self._get_state()
       
        # Episode termination
        done = False
       
        # Info dictionary
        info = {
            'prices': prices.copy(),
            'demands': demands.copy(),
            'shocks': self.current_shocks.copy(),
            'price_indices': self.current_price_idx.copy()  # Include indices for Q-learning
        }
       
        return next_state, profits, done, info

class MarketEnvContinuous(MarketEnv):
    """Market environment that accepts both discrete indices and continuous prices as actions"""
    
    def step(self, actions):
        """Execute one step with flexible action handling
        
        Args:
            actions: List/array where each element can be:
                     - int/np.integer: discrete price index
                     - float: continuous price value
                     
        Returns:
            next_state: Numpy array of actual prices
            profits: Numpy array of profits
            done: Episode termination flag
            info: Dictionary with prices, demands, shocks, and indices
        """
        prices = []
        indices = []
        
        for a in actions:
            if isinstance(a, (int, np.integer)):
                # Discrete action: use price from grid
                prices.append(self.price_grid[a])
                indices.append(a)
            else:
                # Continuous action: use actual price value
                price = float(a)
                prices.append(price)
                # Find closest grid index for tracking
                indices.append(np.argmin(np.abs(self.price_grid - price)))
        
        prices = np.array(prices)
        self.current_price_idx = np.array(indices)
        
        # Generate shocks
        self.current_shocks = self._generate_shocks()
        
        # Calculate demands and profits
        demands, profits = self.calculate_demand_and_profit(prices, self.current_shocks)
        
        # Update time
        self.t += 1
        
        # Get next state (actual prices)
        next_state = self._get_state()
        
        # Episode termination
        done = False
        
        # Info dictionary
        info = {
            'prices': prices.copy(),
            'demands': demands.copy(),
            'shocks': self.current_shocks.copy(),
            'price_indices': self.current_price_idx.copy()
        }
        
        return next_state, profits, done, info

    def get_state_prices(self):
        """Get current state as actual prices (same as _get_state)"""
        return self._get_state()


# `PSOAgent`

In [4]:
class PSOAgent:
    """PSO with full restart when stuck (no improvement for N steps)"""
    def __init__(self, env, agent_id, num_particles=10, w0=0.025, c1=1.75, c2=1.75, 
                 price_min=0.0, price_max=2.0, restart_threshold=300):
        self.env = env
        self.id = agent_id
        self.num_particles = num_particles
        self.w0 = w0
        self.c1 = c1
        self.c2 = c2
        self.price_min = price_min
        self.price_max = price_max
        self.restart_threshold = restart_threshold
        
        # Initialize
        self.particles = np.random.uniform(self.price_min, self.price_max, self.num_particles)
        self.velocities = np.zeros(self.num_particles)
        self.p_best_positions = self.particles.copy()
        self.p_best_scores = np.full(self.num_particles, -np.inf)
        self.g_best_position = np.mean(self.particles)
        self.g_best_score = -np.inf
        
        # Restart tracking
        self.t = 0
        self.stuck_counter = 0
        self.last_g_best = -np.inf
        self.restart_count = 0

    def update(self, opp_price):
        # Inertia weight with floor
        w = max(0.4, 0.9 - 0.5 * self.t / 10000)
        
        # Evaluate particles
        prices = np.full(self.env.n_firms, opp_price)
        scores = np.zeros(self.num_particles)
        for i in range(self.num_particles):
            prices[self.id] = self.particles[i]
            _, profits = self.env.calculate_demand_and_profit(prices, np.zeros(self.env.n_firms))
            scores[i] = profits[self.id]
        
        # Update bests
        better_p = scores > self.p_best_scores
        self.p_best_scores[better_p] = scores[better_p]
        self.p_best_positions[better_p] = self.particles[better_p]
        
        max_score = np.max(scores)
        if max_score > self.g_best_score:
            self.g_best_score = max_score
            self.g_best_position = self.particles[np.argmax(scores)]
        
        # Check if stuck (no significant improvement)
        if self.g_best_score <= self.last_g_best + 1e-6:
            self.stuck_counter += 1
        else:
            self.stuck_counter = 0
            self.last_g_best = self.g_best_score
        
        # RESTART if stuck too long
        if self.stuck_counter >= self.restart_threshold:
            self.particles = np.random.uniform(self.price_min, self.price_max, self.num_particles)
            self.velocities = np.zeros(self.num_particles)
            self.p_best_positions = self.particles.copy()
            self.p_best_scores = np.full(self.num_particles, -np.inf)
            # Keep g_best to remember best ever found
            self.stuck_counter = 0
            self.restart_count += 1
            # if self.restart_count <= 10:  # Only print first 10
                # print(f"      [PSO RESTART #{self.restart_count} at t={self.t}]")
        
        # Update velocities and positions
        r1 = np.random.uniform(0, 1, self.num_particles)
        r2 = np.random.uniform(0, 1, self.num_particles)
        
        self.velocities = (
            w * self.velocities +
            self.c1 * r1 * (self.p_best_positions - self.particles) +
            self.c2 * r2 * (self.g_best_position - self.particles)
        )
        self.velocities = np.clip(self.velocities, -0.3, 0.3)
        self.particles += self.velocities
        self.particles = np.clip(self.particles, self.price_min, self.price_max)
        
        self.t += 1

    def choose_price(self):
        return self.g_best_position


# Simulation

In [5]:
def run_simulation(model, seed, shock_cfg, benchmarks):
    """Run PSO vs PSO simulation"""
    np.random.seed(seed)
    
    env = MarketEnvContinuous(market_model=model, shock_cfg=shock_cfg, seed=seed)
    
    price_min = env.price_grid.min()
    price_max = env.price_grid.max()
    
    agents = [
        PSOAgent(env, agent_id=0, price_min=price_min, price_max=price_max),
        PSOAgent(env, agent_id=1, price_min=price_min, price_max=price_max)
    ]
    
    state = env.reset()
    profits_history = []
    prices_history = []
    
    # Initialize prices
    current_prices = [agents[0].choose_price(), agents[1].choose_price()]
    
    for t in range(env.horizon):
        # Each agent updates with opponent's last price
        agents[0].update(current_prices[1])
        agents[1].update(current_prices[0])
        
        current_prices = [agents[0].choose_price(), agents[1].choose_price()]
        
        next_state, rewards, done, info = env.step(current_prices)
        
        state = next_state
        prices_history.append(current_prices)
        profits_history.append(rewards)
    
    # Calculate averages over last 1000 steps
    last_prices = np.array(prices_history[-1000:])
    avg_price1 = np.mean(last_prices[:, 0])
    avg_price2 = np.mean(last_prices[:, 1])
    
    last_profits = np.array(profits_history[-1000:])
    avg_profit1 = np.mean(last_profits[:, 0])
    avg_profit2 = np.mean(last_profits[:, 1])
    
    # Get benchmarks
    pi_n = benchmarks['E_pi_N']
    pi_m = benchmarks['E_pi_M']
    p_n = benchmarks['p_N']
    p_m = benchmarks['p_M']
    
    # Calculate Delta (profit-based)
    delta1 = (avg_profit1 - pi_n) / (pi_m - pi_n) if (pi_m - pi_n) != 0 else 0
    delta2 = (avg_profit2 - pi_n) / (pi_m - pi_n) if (pi_m - pi_n) != 0 else 0
    
    # Calculate RPDI (pricing-based)
    rpdi1 = (avg_price1 - p_n) / (p_m - p_n) if (p_m - p_n) != 0 else 0
    rpdi2 = (avg_price2 - p_n) / (p_m - p_n) if (p_m - p_n) != 0 else 0
    
    return avg_price1, avg_price2, delta1, delta2, rpdi1, rpdi2, p_n


def main():
    shock_cfg = {
        'enabled': False
    }
    
    benchmark_calculator = TheoreticalBenchmarks(seed=SEED)
    
    print("=" * 80)
    print("PSO vs PSO - SCHEME NONE")
    print("=" * 80)
    
    all_benchmarks = benchmark_calculator.calculate_all_benchmarks(shock_cfg)
    
    models = ['logit', 'hotelling', 'linear']
    results = {}
    
    for model in models:
        print(f"\nRunning {model.upper()} simulations...")
        
        model_benchmarks = all_benchmarks[model]
        
        avg_prices1 = []
        avg_prices2 = []
        deltas1 = []
        deltas2 = []
        rpdis1 = []
        rpdis2 = []
        theo_prices = []
        
        for run in range(NUM_RUNS):
            seed = SEED + run
            ap1, ap2, d1, d2, r1, r2, p_n = run_simulation(model, seed, shock_cfg, model_benchmarks)
            avg_prices1.append(ap1)
            avg_prices2.append(ap2)
            deltas1.append(d1)
            deltas2.append(d2)
            rpdis1.append(r1)
            rpdis2.append(r2)
            theo_prices.append(p_n)
        
        results[model] = {
            'Avg Price Firm 1': np.mean(avg_prices1),
            'Theo Price': np.mean(theo_prices),
            'Avg Price Firm 2': np.mean(avg_prices2),
            'Delta 1': np.mean(deltas1),
            'Delta 2': np.mean(deltas2),
            'RPDI 1': np.mean(rpdis1),
            'RPDI 2': np.mean(rpdis2)
        }
        
        print(f"  Completed: Firm 1 Δ = {results[model]['Delta 1']:.3f}, Firm 2 Δ = {results[model]['Delta 2']:.3f}")
    
    data = {
        'Model': [m.upper() for m in models],
        'Firm 1 Avg. Prices': [round(results[m]['Avg Price Firm 1'], 2) for m in models],
        'Theo. Prices': [round(results[m]['Theo Price'], 2) for m in models],
        'Firm 2 Avg. Prices': [round(results[m]['Avg Price Firm 2'], 2) for m in models],
        'Theo. Prices ': [round(results[m]['Theo Price'], 2) for m in models],
        'Firm 1 Extra-profits Δ': [round(results[m]['Delta 1'], 2) for m in models],
        'Firm 2 Extra-profits Δ': [round(results[m]['Delta 2'], 2) for m in models],
        'Firm 1 RPDI': [round(results[m]['RPDI 1'], 2) for m in models],
        'Firm 2 RPDI': [round(results[m]['RPDI 2'], 2) for m in models]
    }
    
    df = pd.DataFrame(data)
    df.to_csv("./results/pso_vs_pso.csv", index=False)
    
    print("\n" + "=" * 80)
    print("FINAL RESULTS")
    print("=" * 80)
    print(df.to_string(index=False))
    
    # Overall averages
    print("\n" + "=" * 80)
    print("OVERALL AVERAGES ACROSS ALL MODELS")
    print("=" * 80)
    
    avg_delta1 = np.mean([results[m]['Delta 1'] for m in models])
    avg_delta2 = np.mean([results[m]['Delta 2'] for m in models])
    avg_rpdi1 = np.mean([results[m]['RPDI 1'] for m in models])
    avg_rpdi2 = np.mean([results[m]['RPDI 2'] for m in models])
    
    print("\nFirm 1 (PSO):")
    print(f"  Average Delta (Δ): {avg_delta1:.4f}")
    print(f"  Average RPDI:      {avg_rpdi1:.4f}")
    print("\nFirm 2 (PSO):")
    print(f"  Average Delta (Δ): {avg_delta2:.4f}")
    print(f"  Average RPDI:      {avg_rpdi2:.4f}")
    
    print("\n[Results saved to ./results/pso_vs_pso.csv]")


if __name__ == "__main__":
    main()


PSO vs PSO - SCHEME NONE

Running LOGIT simulations...


KeyboardInterrupt: 